<!---
title: 'Large Language Model Prompt Engineering for Complex Summarization'
author: John Stewart
slug: gpt-summary-prompt-engineering
post_date: 2023-03-26 12:01:00
categories: GPT, LLM, Prompt-Engineering, Medical
tags: GPT, LLM, prompt-engineering, medical
# featured_image: assets/pair-programming-raccoons.png
summary: Learn how to use GPT / LLMs to create complex summaries such as for medical text
--->

<style>
.output {
    font-size: 30%;
}
</style>

# Large Language Model Prompt Engineering for Complex Summarization 
The recent explosion in the popularity of Large language Models (LLM) such as ChatGPT has opened the floodgates to an enormous and ever-growing list of possible new applications in a lot of different fields. 
On a recent engagement, our team created a demo of how to use the [Azure OpenAI]() service to leverage LLM capabilities in generating summaries of medical documents for non-specialist readers.

## Background
Every day, hundreds of new medical specialist papers are published on sites such as [PubMed](). 
For patients or caregivers with a keen interest in new research impacting their condition, it can often be difficult to comphrehend the complex jargon and language.
Consequently, many journals are require submitters to produce a separate, short [Plain Language Summary]() for the non-specialist reader. 
Our customer asked that we prototype a way to use GPT to produce these lay-summaries using AI, freeing up 
time for researchers and editors to concentrate on their primary focus of publishing new medical research.

## Hypothesis
A model like OpenAI's Davinci-3, the original LLM that underpinned ChatGPT, could produce a passable Plain Language Summary of medical text describing a drug-study, which could then be refined by an author or editor in short time.  
We targed a complete summary, including important details from the source text like patient population, treatment outcomes, and how the research impacted disease treatment.  
The output needed to be ~250 words, omit specialist medical terms, and be informative enough for the reader to get a full understanding of the source paper.
## Setup
We are using the [LangChain](https://python.langchain.com/en/latest/index.html) python library as a harness for our use of Azure OpenAI and GPT3.  
Ensure you have a new virtual-environment setup and install the needed dependencies by running `pip install -r requirements.txt` from the root of the Github project.
We will use the [pdfminer](https://pypi.org/project/pdfminer/) library to convert the source paper PDF into plaintext for ingestion into Azure OpenAI GPT.
The prompt-engineering exercise uses a representative medical article from the [Journal of Rare Diseases](https://ojrd.biomedcentral.com/counter/pdf/10.1186/1750-1172-10-S1-P8.pdf).  

In [52]:
import os
from dotenv import load_dotenv 
load_dotenv() # make sure to set your Azure OpenAI keys below in your own .env file
DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")

from langchain.llms import AzureOpenAI
from langchain.document_loaders import TextLoader
from pdfminer.high_level import extract_text
from langchain import PromptTemplate, LLMChain
from langchain.chains import LLMChain

llm = AzureOpenAI(deployment_name=DEPLOYMENT_NAME, model_name="text-davinci-003", max_tokens=500)

study_txt = TextLoader('./bleegblorgimab.txt').load()[0].page_content

#### Initial basic "TLDR;" prompt
Starting with a simple command to GPT.  Given the GPT models are trained to follow instructions it should 'know' what to do... 

In [53]:
prompt_template = """
Write a Plain Language Summary of the medical study below:
{study_text}
"""
chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template))
summary = chain.run(study_txt)
print(summary)
# count the number of words in the summary
print(f"PLS length: {len(summary.split())}")


This open-label study evaluated the efficacy and safety of Bleegblorgumab, a monoclonal antibody specifically designed to target HER2, in combination with Prednisolone in 50 HER2+ metastatic breast cancer (MBC) patients who had received prior HER2-directed therapy. The results showed that the combination was generally well-tolerated with fatique, nausea and peripheral neuropathy being the most common adverse events. The overall response rate (ORR) was 70%, with 35 patients achieving partial response and 7 patients achieving complete response. The median progression-free survival (PFS) was 12.5 months, and the median overall survival (OS) was not reached at the time of data analysis. These findings suggest that Bleegblorgumab in combination with Prednisolone may offer a valuable therapeutic option for HER2+ MBC patients who have previously received HER2-directed therapy. However, larger randomized controlled trials are needed to confirm these findings and further evaluate the long-term

This is a good start, but the language is way too technical.  Also we wanted ~ 250 words so it needs to be longer. OpenAI suggests you be as explicit as possible in your prompt-commands.  Let's get more explicit!

In [54]:
prompt_template = """
Summarize the medical text below for a layperson. Simplify all medical jargon to plain language. 
Write 250 words at a 6th grade reading level.
{study_text}
"""

chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template))
summary = chain.run(study_txt)
print(summary)


This study looked at the combination of Bleegblorgumab and Prednisolone in 50 people with HER2+ metastatic breast cancer (MBC). HER2+ MBC is a type of breast cancer that is aggressive and has a poor prognosis. Bleegblorgumab is a medicine that targets HER2. The study looked at how well the combination of Bleegblorgumab and Prednisolone worked, how long it worked and how safe it was. The results showed that the combination worked well, with 70% of people having a partial or complete response, and 92% having disease control. The median progression-free survival was 12.5 months and the median overall survival had not been reached at the time of this study. Side effects were mostly mild to moderate and included fatigue, nausea, and peripheral neuropathy. The results of this study show that the combination of Bleegblorgumab and Prednisolone may provide a valuable therapeutic option for people with HER2+ MBC and further studies are needed to confirm the findings and evaluate the long-term s

Ok, we reduced the amount of 'medical jargon', but we didnt get a more complete summary we got LESS complete.  :(  Lets be explicit about what we expect in the output...  

In [55]:
prompt_template = """
{study_text}

Write a Plain Language Summary of the above medical study for a layperson.
Translate any medical terms to simple english explanations.
Include the following:
- What was the purpose of the study?
- What did the researchers do?
- What did they find?
- What does this mean for me?
Write 250 words at a 6th grade reading level.
"""

chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template))
summary = chain.run(study_txt)
print(summary)


This study looked at the use of a new drug combination to treat metastatic breast cancer (MBC) that is HER2-positive. HER2-positive MBC is a type of breast cancer that is aggressive and hard to treat. The drug combination used in this study was Bleegblorgumab and Prednisolone. Bleegblorgumab is a monoclonal antibody specifically designed to target HER2, while Prednisolone is an immunomodulatory medication. The researchers wanted to find out if this drug combination would be effective in treating HER2-positive MBC.

The researchers enrolled fifty patients with HER2-positive MBC who had received prior HER2-directed therapy. The patients received Bleegblorgumab intravenously at a dose of 10 mg/kg every three weeks, and Prednisolone orally at 10 mg/day. The researchers monitored the patients regularly to assess the response to the treatment. The primary endpoint of the study was the overall response rate (ORR), while the secondary endpoints included progression-free survival (PFS), overal

This is quite a bit better!  It includes some context from the patient-perspective.  It's got a nice disclaimer at the end.  

One thing to note is we're now including the source study text _above_ our prompt.  This helps with the [recency problem](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/concepts/advanced-prompt-engineering?pivots=programming-language-chat-completions#repeat-instructions-at-the-end) with LLMs.  Putting the prompt last helps the LLM to stay on task.

Lets do a bit more fine-tuning.  We want the output to be in first-person or "active voice".  We also would like it to use bullet-points instead of paragraph style.  Lets see what we can do...

In [56]:
prompt_template = """
You are a medical researcher writing a Plain Language Summary of your study for a layperson.
{study_text}
Write a Plain Language Summary of the above medical study for a layperson.
Translate any medical terms to simple english explanations.
Use first-person 'We'.  Use short bullet points.
Answer these questions:
- What was the purpose of the study?
- What did the researchers do?
- What did they find?
- What does this mean for me?
Write 250 words at a 6th grade reading level.
"""

chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template))
summary = chain.run(study_txt)
print(summary)


We conducted a study to find out if a new treatment, Bleegblorgumab in combination with Prednisolone, could help people with a specific type of breast cancer called HER2+ MBC. HER2+ MBC is a type of breast cancer that is more aggressive and has a poorer prognosis than other types of breast cancer. 

We enrolled 50 people with HER2+ MBC who had already tried other treatments for their cancer. We gave Bleegblorgumab intravenously every three weeks, and Prednisolone orally every day. We then monitored the participants for any changes in their cancer and side effects. 

We found that the combination of Bleegblorgumab and Prednisolone was effective in treating HER2+ MBC. 70% of patients achieved either a complete or partial response to the treatment, and 92% experienced disease control. The median progression-free survival was 12.5 months, and the median overall survival was not reached at the time of data analysis, which suggests a potential survival benefit. The treatment was generally w

#### Good but not perfect.  
We got first-person but no bullet-points!  

There's a lot more we could do here in terms of both engineering the inital prompt, as well as using a multi-step 'chain' of LLM calls to produce the exact output we want.  We'll save that for our next blog post!

### Caveats
#### Length of paper
The Azure OpenAI Davinci-3 model has a [limit](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/concepts/models#model-summary-table-and-region-availability) of 4097 [tokens](https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them) of combined input and output.  
Given that we want around 250 words of output and a token represents say one-half of a word, we will reserve 500 tokens for the completion response.  That leaves ~3500 tokens for input, or about 1700 words.
Any source text longer than 1700 words will need to be manually edited to fit.
#### Graphs and Figures
The Davinci-3 model was trained on billions of words of text, including a lot of graphs and charts.  However we can only input plain-text into the OpenAI service.
For this experiment we will assume the input is only plain-text, and whatever unformatted table data that can be extracted automatically from the source PDF.
#### Need for Human Review
It bears repeating that the goal of this experiment is to make a _draft_ summary, and any output produced must be reviewed, edited, and approved by a responsible human party.

### Future Directions

In the future we hope to bring you examples of managing longer text input, usage of GPT4, and chat-based interactions for fine-tuning the summarization output.  As well we may be able to exploit GPT4's abilities to 'understand' charts, graphs and tables.

### Acknowledgements
Authored by [John Stewart](https://www.linkedin.com/in/john-ms/), Microsoft Commercial Software Engineering, with helpful editing and suggestions from ChatGPT.